In [14]:
import os
import glob
import json
import numpy as np
import pandas as pd
from tqdm import tqdm

In [18]:
from kobart import get_kobart_tokenizer
kobart_tokenizer = get_kobart_tokenizer()
kobart_tokenizer.tokenize("안녕하세요. 한국어 BART 입니다.🤣:)l^o")

using cached model


['▁안녕하', '세요.', '▁한국어', '▁B', 'A', 'R', 'T', '▁입', '니다.', '🤣', ':)', 'l^o']

In [10]:
RAW_DATA_DIR = 'datasets/daicon'

In [15]:
with open(RAW_DATA_DIR+'/train.jsonl', 'r', encoding='UTF-8') as json_file:
    json_list = list(json_file)

trains = []
for json_str in json_list:
    line = json.loads(json_str)
    trains.append(line)

train_df = pd.DataFrame(trains)
train_df.head()

,media,id,article_original,abstractive,extractive
0,당진시대,327827480,[당진시 문화관광과를 대상으로 하는 행정사무감사에서 당진시립합창단 관계자가 보낸 것...,지난 6일 당진시의회 행정사무감사에서 '합창단이 소리를 작게 낼 것이니 알고 있으라...,"[0, 1, 2]"
1,국제신문,339840364,[미국 메이저리그(MLB)에서 활동하는 한국 선수들의 시즌 초반 희비가 엇갈리고 있...,LA 에인절스의 최지만이 맹활약을 하여 시즌 타율 0.250에서 0.313으로 올리...,"[0, 1, 3]"
2,기호일보,371071597,"[인천 부영공원 운영 생활야구협회 80여 동호회 팀에 260만 원씩 받아, 국유지로...",16일 부평구와 협회 등에 따르면 부영공원 안에 있는 야구장을 구생활체육야구협회가 ...,"[6, 7, 10]"
3,대구일보,354806783,[대구·경북첨단의료산업진흥재단 의약생산센터는 항암주사제 무균충전 시설을 갖추고 있다...,대구·경북첨단의료산업진흥재단 의약생산센터는 약사법 시행규칙에서 정한 바에 따라 전용...,"[1, 6, 4]"
4,대구신문,347022773,[국내 유통되는 탄산음료 중 식품의약품안전처에 품질인증을 받은 제품이 하나도 없는 ...,식품의약품안전처는 29일 어린이가 즐겨마시는 음료를 대상으로 영양성분을 조사한 결과...,"[2, 3, 10]"


In [16]:
train_df['article_original'][1]

['미국 메이저리그(MLB)에서 활동하는 한국 선수들의 시즌 초반 희비가 엇갈리고 있다.',
 'LA 에인절스의 최지만은 맹활약으로 주전 입지를 굳혀가는 반면 텍사스 레인저스의 추신수는 개막전 선발에서 밀려나는 수모를 당한 뒤에서도 좀처럼 반등을 하지 못하고 있다.',
 '최지만은 2일(한국시간) 미국 플로리다주 세인트피터즈버그 트로피카나 필드에서 열린 2019 미국프로야구 메이저리그 콜로라도 로키스와 홈경기에 3번 타자 1루수로 선발 출전해 4타수 2안타를 치고 1득점을 올렸다.',
 '시즌 타율은 0.250에서 0.313(16타수 5안타)로 올랐다.',
 '최지만은 개막전에서 4타수 무안타로 침묵했으나 이후 3경기 연속 안타이자 2경기 연속 멀티 히트를 뽑아내며 입지를 굳혀갔다.',
 '2016년 LA 에인절스 소속으로 메이저리그에 데뷔한 최지만이 2경기 연속 멀티 히트를 친 것은 이번이 처음이다.',
 '스코어가 벌어진 탓에 콜로라도 불펜의 필승조인 오승환과 최지만의 맞대결은 성사되지 않았다.',
 '추신수는 들쭉날쭉한 출전 일정 탓인지 타격감을 찾지 못하고 있다.',
 '추신수는 이날 텍사스주 알링턴 글로브 라이프 파크에서 열린 휴스턴 애스트로스와 홈경기에 1번 지명타자로 선발 출전했지만 4타수 무안타로 침묵했다.',
 '시즌 타율은 0.167에서 0.100(10타수 1안타)로 떨어졌다.',
 '추신수는 11년 만에 개막전 선발 라인업에서 제외되는 등 이날까지 텍사스가 치른 4경기 가운데 2경기에만 선발 출전했다.',
 '크리스 우드워드 텍사스 감독은 상대 선발이 좌완이면 좌타자인 추신수 대신 우타자인 헌터 펜스를 선발 지명타자로 기용하고 있다.',
 '텍사스(2승 2패)는 휴스턴 마운드에 2안타로 꽁꽁 묶이며 1-2로 패해 2연승 행진을 마감했다.',
 '한편 피츠버그 파이리츠의 강정호는 914일 만에 홈경기에 나섰지만 안타를 만들어내지 못했다.',
 '이날 펜실베이니아주 피츠버그의 PNC 파크에서 열린 세인트루이스 카디널스와 홈 개막전에 선발 라인업

In [19]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42803 entries, 0 to 42802
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   media             42803 non-null  object
 1   id                42803 non-null  object
 2   article_original  42803 non-null  object
 3   abstractive       42803 non-null  object
 4   extractive        42803 non-null  object
dtypes: object(5)
memory usage: 1.6+ MB
